<a href="https://colab.research.google.com/github/natasharavinand/Amity/blob/main/sentiment_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Loading in Data**


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import time

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
indicator_tweets = pd.read_csv('/content/drive/MyDrive/makeathon_stuff/suicide_tweets_df.csv')

full_sentiment = pd.read_csv('/content/drive/MyDrive/makeathon_stuff/sentiment_full.csv', encoding="ISO-8859-1", names=["0", "1", "2", "3", "4", "5"])

In [ ]:
full_sentiment

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
reduced_sentiment = full_sentiment.iloc[::20, :]
reduced_sentiment

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
20,0,1467813985,Mon Apr 06 22:20:37 PDT 2009,NO_QUERY,quanvu,@alydesigns i was out most of the day so didn'...
40,0,1467818020,Mon Apr 06 22:21:39 PDT 2009,NO_QUERY,itsanimesh,really don't feel like getting up today... but...
60,0,1467822687,Mon Apr 06 22:22:52 PDT 2009,NO_QUERY,xVivaLaJuicyx,"@BatManYNG I miss my ps3, it's out of commissi..."
80,0,1467834053,Mon Apr 06 22:25:52 PDT 2009,NO_QUERY,thelazyboy,sleep soon... i just hate saying bye and see y...
...,...,...,...,...,...,...
1599900,4,2193574897,Tue Jun 16 08:38:37 PDT 2009,NO_QUERY,johnwelshphd,@cris1015 My goal on stocks like $MTXX is to h...
1599920,4,2193575839,Tue Jun 16 08:38:42 PDT 2009,NO_QUERY,bendotorg,Largest/fastest hotspot in U.S.A. is located i...
1599940,4,2193576797,Tue Jun 16 08:38:47 PDT 2009,NO_QUERY,jaycemiguel,"@MsKCJones Yeah it's very irritating, right? B..."
1599960,4,2193577726,Tue Jun 16 08:38:52 PDT 2009,NO_QUERY,FrayBaby,@pokapolas love the donut and the toadstool.


In [ ]:
print(indicator_tweets.dtypes)
indicator_tweets.head(4)

created_at         object
text               object
user_name          object
screen_name        object
location           object
friends_count       int64
followers_count     int64
is_quote_status      bool
retweet_count       int64
favorite_count      int64
retweeted            bool
favorited            bool
verified             bool
protected            bool
lang               object
dtype: object


,created_at,text,user_name,screen_name,location,friends_count,followers_count,is_quote_status,retweet_count,favorite_count,retweeted,favorited,verified,protected,lang
0,2021-03-12 04:57:25,RT @FatimaZ53524989: Who will commit after bre...,"i am not a bot, rot in hell bollywood!",BhawnaBatra20,NaN,578,382,False,14,0,False,False,False,False,en
1,2021-03-12 04:57:24,@CharitieHartsig I think he was allowed to com...,Nick Danger,NickDan56654107,NaN,139,45,False,0,0,False,False,False,False,en
2,2021-03-12 04:57:08,RT @Taman_Rupal: “The farmers commit suicide s...,jashan ਸਿੰਘ,jashansl12,NaN,548,501,False,256,0,False,False,False,False,en
3,2021-03-12 04:56:59,RT @Ayam_July: Piano wants to commit suicide o...,1Reu_Ben,gyama_wagyimi,kumerica,5034,4768,False,130,0,False,False,False,False,en


In [ ]:
reduced_sentiment.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
20,0,1467813985,Mon Apr 06 22:20:37 PDT 2009,NO_QUERY,quanvu,@alydesigns i was out most of the day so didn'...
40,0,1467818020,Mon Apr 06 22:21:39 PDT 2009,NO_QUERY,itsanimesh,really don't feel like getting up today... but...
60,0,1467822687,Mon Apr 06 22:22:52 PDT 2009,NO_QUERY,xVivaLaJuicyx,"@BatManYNG I miss my ps3, it's out of commissi..."
80,0,1467834053,Mon Apr 06 22:25:52 PDT 2009,NO_QUERY,thelazyboy,sleep soon... i just hate saying bye and see y...


# **Cleaning/Adding Various Column**

In [ ]:
reduced_sentiment[reduced_sentiment['3'] != 'NO_QUERY']
reduced_sentiment = reduced_sentiment.drop(columns = ['3'])
reduced_sentiment = reduced_sentiment.rename(columns={'0': 'polarity', '1': 'user_id', '2': 'created_at', '4': 'user_name', '5': 'tweet'})
reduced_sentiment.head()

,polarity,user_id,created_at,user_name,tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
20,0,1467813985,Mon Apr 06 22:20:37 PDT 2009,quanvu,@alydesigns i was out most of the day so didn'...
40,0,1467818020,Mon Apr 06 22:21:39 PDT 2009,itsanimesh,really don't feel like getting up today... but...
60,0,1467822687,Mon Apr 06 22:22:52 PDT 2009,xVivaLaJuicyx,"@BatManYNG I miss my ps3, it's out of commissi..."
80,0,1467834053,Mon Apr 06 22:25:52 PDT 2009,thelazyboy,sleep soon... i just hate saying bye and see y...


In [ ]:
import re 

regex_link= '(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?'

# reduced_sentiment['link'] = 
# reduced_sentiment['tweet'].iloc[.str.extract(regex_link, expand=False)
# r1 = reduced_sentiment['tweet'].iloc[0]
# re.search(regex_link,r1).group()

def extract_regex(regex,tweet):
  if re.search(regex,tweet) is not None:
    return re.search(regex,tweet).group() 
  else:
    return np.nan

reduced_sentiment['link'] = reduced_sentiment['tweet'].apply(lambda x: extract_regex(regex_link, x))

In [ ]:
reduced_sentiment_1 = reduced_sentiment.copy()
reduced_sentiment_1['tweet'] = reduced_sentiment_1['tweet'].str.replace(regex_link, '')
reduced_sentiment_1

,polarity,user_id,created_at,user_name,tweet,link
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,_TheSpecialOne_,"@switchfoot - Awww, that's a bummer. You sho...",http://twitpic.com/2y1zl
20,0,1467813985,Mon Apr 06 22:20:37 PDT 2009,quanvu,@alydesigns i was out most of the day so didn'...,NaN
40,0,1467818020,Mon Apr 06 22:21:39 PDT 2009,itsanimesh,really don't feel like getting up today... but...,NaN
60,0,1467822687,Mon Apr 06 22:22:52 PDT 2009,xVivaLaJuicyx,"@BatManYNG I miss my ps3, it's out of commissi...",NaN
80,0,1467834053,Mon Apr 06 22:25:52 PDT 2009,thelazyboy,sleep soon... i just hate saying bye and see y...,NaN
...,...,...,...,...,...,...
1599900,4,2193574897,Tue Jun 16 08:38:37 PDT 2009,johnwelshphd,@cris1015 My goal on stocks like $MTXX is to h...,NaN
1599920,4,2193575839,Tue Jun 16 08:38:42 PDT 2009,bendotorg,Largest/fastest hotspot in U.S.A. is located i...,http://bit.ly/eDjz7
1599940,4,2193576797,Tue Jun 16 08:38:47 PDT 2009,jaycemiguel,"@MsKCJones Yeah it's very irritating, right? B...",NaN
1599960,4,2193577726,Tue Jun 16 08:38:52 PDT 2009,FrayBaby,@pokapolas love the donut and the toadstool.,NaN


In [ ]:
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
months = ['Apr','May','Jun']
days_regex = "(" + ")|(".join(days) + ")"
months_regex = "(" + ")|(".join(months) + ")"

re.search(days_regex,reduced_sentiment_1['created_at'].iloc[0]).group

reduced_sentiment_1['weekday'] = reduced_sentiment_1['created_at'].apply(lambda x: extract_regex(days_regex, x))
reduced_sentiment_1['month'] = reduced_sentiment_1['created_at'].apply(lambda x: extract_regex(months_regex, x))


In [ ]:
reduced_sentiment_2 = reduced_sentiment_1.copy()
reduced_sentiment_2['created_at'] = reduced_sentiment_2['created_at'].str.replace(days_regex, '')
reduced_sentiment_2['created_at'] = reduced_sentiment_2['created_at'].str.replace(months_regex, '')
reduced_sentiment_2['created_at'] = reduced_sentiment_2['created_at'].str.replace('PDT', '')
reduced_sentiment_2['created_at'] = reduced_sentiment_2['created_at'].str.replace('2009', '')
reduced_sentiment_2['created_at'] = reduced_sentiment_2['created_at'].str.replace('2009', '')

reduced_sentiment_2['day'] = reduced_sentiment_2['created_at'].str[2:4]
reduced_sentiment_2['created_at'] = reduced_sentiment_2['created_at'].str[4:]
reduced_sentiment_2['created_at'] = reduced_sentiment_2['created_at'].str.strip()
reduced_sentiment_2


,polarity,user_id,created_at,user_name,tweet,link,weekday,month,day
0,0,1467810369,22:19:45,_TheSpecialOne_,"@switchfoot - Awww, that's a bummer. You sho...",http://twitpic.com/2y1zl,Mon,Apr,06
20,0,1467813985,22:20:37,quanvu,@alydesigns i was out most of the day so didn'...,NaN,Mon,Apr,06
40,0,1467818020,22:21:39,itsanimesh,really don't feel like getting up today... but...,NaN,Mon,Apr,06
60,0,1467822687,22:22:52,xVivaLaJuicyx,"@BatManYNG I miss my ps3, it's out of commissi...",NaN,Mon,Apr,06
80,0,1467834053,22:25:52,thelazyboy,sleep soon... i just hate saying bye and see y...,NaN,Mon,Apr,06
...,...,...,...,...,...,...,...,...,...
1599900,4,2193574897,08:38:37,johnwelshphd,@cris1015 My goal on stocks like $MTXX is to h...,NaN,Tue,Jun,16
1599920,4,2193575839,08:38:42,bendotorg,Largest/fastest hotspot in U.S.A. is located i...,http://bit.ly/eDjz7,Tue,Jun,16
1599940,4,2193576797,08:38:47,jaycemiguel,"@MsKCJones Yeah it's very irritating, right? B...",NaN,Tue,Jun,16
1599960,4,2193577726,08:38:52,FrayBaby,@pokapolas love the donut and the toadstool.,NaN,Tue,Jun,16


In [ ]:
reduced_sentiment_2['created_at'].datetime.strftime(%H%M%S)

SyntaxError: ignored